In [2]:
from pathlib import Path
import json
import pandas as pd

# Map Trnsactions

In [3]:
def Map_Transactions():
    root_dir = Path("../data/map/transaction/hover/country/india/state")

    lis = []
    for i in root_dir.rglob("*.json"):
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)

            state = i.parent.parent.name
            year = i.parent.name

            quarter = i.stem
            quarter = f'Q{quarter}'
            

            lis.append({'state':state, 'year':year, 'quarter':quarter, 'data':dataset})

    # print(lis)

    lis2=[]
    for i in lis:
        for j in i['data']['data']['hoverDataList']:
            # amount format with 2 decimals
            amount_value = j['metric'][0]['amount']
            amount_formatted = f'{amount_value: .2f}'

            data = dict(quarter = i['quarter'],
                        year = i['year'],
                        state = i['state'],
                        district_name = j['name'],
                        type =j['metric'][0]['type'],
                        count = j['metric'][0]['count'],
                        amount = amount_formatted,
                        timestamp = i['data']['responseTimestamp'])
            
            lis2.append(data)

    return lis2

In [21]:
MapTransactions = pd.DataFrame(Map_Transactions())
# MapTransactions.iloc[50:101]
MapTransactions['year'] = MapTransactions['year'].astype(int)
MapTransactions['amount'] = MapTransactions['amount'].astype(float)
MapTransactions['timestamp'] = pd.to_datetime(MapTransactions['timestamp'], unit='ms')
MapTransactions['timestamp'] = MapTransactions['timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")
# MapTransactions['amount'] = pd.to_numeric(MapTransactions['amount'], errors='coerce')
# MapTransactions['amount'] = MapTransactions['amount'].round(2)
MapTransactions

,quarter,year,state,district_name,type,count,amount,timestamp
0,Q1,2018,andaman-&-nicobar-islands,north and middle andaman district,TOTAL,442,9.316631e+05,2021-09-01 13:28:30
1,Q1,2018,andaman-&-nicobar-islands,south andaman district,TOTAL,5688,1.256025e+07,2021-09-01 13:28:30
2,Q1,2018,andaman-&-nicobar-islands,nicobars district,TOTAL,528,1.139849e+06,2021-09-01 13:28:30
3,Q2,2018,andaman-&-nicobar-islands,north and middle andaman district,TOTAL,825,1.317863e+06,2021-09-01 13:28:30
4,Q2,2018,andaman-&-nicobar-islands,south andaman district,TOTAL,9395,2.394824e+07,2021-09-01 13:28:30
...,...,...,...,...,...,...,...,...
16095,Q2,2023,west-bengal,nadia district,TOTAL,26194658,5.410386e+10,2023-08-21 12:00:57
16096,Q2,2023,west-bengal,birbhum district,TOTAL,14912523,2.941312e+10,2023-08-21 12:00:57
16097,Q2,2023,west-bengal,purba medinipur district,TOTAL,30724502,6.595899e+10,2023-08-21 12:00:57
16098,Q2,2023,west-bengal,maldah district,TOTAL,24677357,5.218621e+10,2023-08-21 12:00:57


In [30]:
import mysql.connector
from tabulate import tabulate

In [35]:
db = mysql.connector.connect(
    host ="localhost",
    user = "root",
    password = "balaji",
    database = 'pulse'
)

mycursor = db.cursor(buffered=True)
mycursor

In [38]:
MapTransactions

,quarter,year,state,district_name,type,count,amount,timestamp
0,Q1,2018,andaman-&-nicobar-islands,north and middle andaman district,TOTAL,442,9.316631e+05,2021-09-01 13:28:30
1,Q1,2018,andaman-&-nicobar-islands,south andaman district,TOTAL,5688,1.256025e+07,2021-09-01 13:28:30
2,Q1,2018,andaman-&-nicobar-islands,nicobars district,TOTAL,528,1.139849e+06,2021-09-01 13:28:30
3,Q2,2018,andaman-&-nicobar-islands,north and middle andaman district,TOTAL,825,1.317863e+06,2021-09-01 13:28:30
4,Q2,2018,andaman-&-nicobar-islands,south andaman district,TOTAL,9395,2.394824e+07,2021-09-01 13:28:30
...,...,...,...,...,...,...,...,...
16095,Q2,2023,west-bengal,nadia district,TOTAL,26194658,5.410386e+10,2023-08-21 12:00:57
16096,Q2,2023,west-bengal,birbhum district,TOTAL,14912523,2.941312e+10,2023-08-21 12:00:57
16097,Q2,2023,west-bengal,purba medinipur district,TOTAL,30724502,6.595899e+10,2023-08-21 12:00:57
16098,Q2,2023,west-bengal,maldah district,TOTAL,24677357,5.218621e+10,2023-08-21 12:00:57


In [26]:
mycursor.execute("""create table if not exists mapdataTrans (
                 mapDataTransId int auto_increment primary key, quarter varchar(10), year int(10),
                 state varchar(255), district_name varchar(255), type varchar(10), count int(255),
                 amount float, timestamp datetime) """)

In [37]:
mycursor.execute("show tables")

out = mycursor.fetchall()
print(tabulate(out, [i[0] for i in mycursor.description], tablefmt='psql'))

+-------------------+
| Tables_in_pulse   |
|-------------------|
| aggdatatrans      |
| aggdatausers      |
| mapdatatrans      |
+-------------------+


In [39]:
sql = ("""insert into mapdatatrans (mapDataTransId, quarter, year, state, district_name, type, count,
       amount, timestamp) values (%s, %s, %s, %s, %s, %s, %s, %s, %s)
       on duplicate key update
       quarter = values(quarter), year= values(year), state = values(state), 
       district_name = values(district_name), type = values(type), count = values(count), 
       amount = values(amount), timestamp = values(timestamp)""")

for i in MapTransactions.to_records().tolist():
    mycursor.execute(sql, i)

In [41]:
db.commit()

In [40]:
mycursor.execute("select * from mapdatatrans limit 10")

out= mycursor.fetchall()
print(tabulate(out, [i[0] for i in mycursor.description], tablefmt='psql'))

+------------------+-----------+--------+---------------------------+-----------------------------------+--------+---------+-------------+---------------------+
|   mapDataTransId | quarter   |   year | state                     | district_name                     | type   |   count |      amount | timestamp           |
|------------------+-----------+--------+---------------------------+-----------------------------------+--------+---------+-------------+---------------------|
|                1 | Q1        |   2018 | andaman-&-nicobar-islands | south andaman district            | TOTAL  |    5688 | 1.25602e+07 | 2021-09-01 13:28:30 |
|                2 | Q1        |   2018 | andaman-&-nicobar-islands | nicobars district                 | TOTAL  |     528 | 1.13985e+06 | 2021-09-01 13:28:30 |
|                3 | Q2        |   2018 | andaman-&-nicobar-islands | north and middle andaman district | TOTAL  |     825 | 1.31786e+06 | 2021-09-01 13:28:30 |
|                4 | Q2        |  

# Map Users

In [43]:
def Map_Users():
    root_dir = Path("../data/map/user/hover/country/india/state")

    lis = []
    for i in root_dir.rglob("*.json"):
        with open(i, "r") as file:
            context = file.read()
            dataset = json.loads(context)

            state = i.parent.parent.name
            year = i.parent.name

            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({'state':state, 'year': year, 'quarter':quarter, 'data':dataset})

    lis2 = []
    for i in lis:
        hoverData = i['data']['data']['hoverData']
        # .items() is used to iterate key value pairs
        # district_name is key and district data is value
        
        for district_name, district_data in hoverData.items():
          
            data = dict(quarter = i['quarter'],
                        year = i['year'],
                        state = i['state'],
                        disrict_name = district_name,
                        registeredUsers = district_data['registeredUsers'],
                        appOpens = district_data['appOpens'],
                        timestamp =i['data']['responseTimestamp'])


            lis2.append(data)

    return lis2

In [47]:
MapUsers = pd.DataFrame(Map_Users())
MapUsers['year'] = MapUsers['year'].astype(int)
MapUsers['timestamp'] = pd.to_datetime(MapUsers['timestamp'], unit='ms')
MapUsers['timestamp'] = MapUsers['timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")
MapUsers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16104 entries, 0 to 16103
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   quarter          16104 non-null  object
 1   year             16104 non-null  int32 
 2   state            16104 non-null  object
 3   disrict_name     16104 non-null  object
 4   registeredUsers  16104 non-null  int64 
 5   appOpens         16104 non-null  int64 
 6   timestamp        16104 non-null  object
dtypes: int32(1), int64(2), object(4)
memory usage: 817.9+ KB


In [48]:
mycursor.execute("""create table if not exists mapdatausers (
                 mapDataUsersId int auto_increment primary key, quarter varchar(10), year int(10),
                 state varchar(255), district_name varchar(255), registeredUsers bigint, 
                 appOpens bigint, timestamp datetime)""")

In [50]:
sql = ("""insert into mapdatausers (mapDataUsersId, quarter, year, state, district_name, 
       registeredUsers, appOpens, timestamp) values(%s, %s, %s, %s, %s, %s, %s, %s)
       on duplicate key update
       quarter = values(quarter), year = values(year), state = values(state), 
       district_name = values(district_name), registeredUsers = values(registeredUsers), 
       appOpens = values(appOpens), timestamp = values(timestamp)""")

for i in MapUsers.to_records().tolist():
    mycursor.execute(sql, i)

In [51]:
db.commit()